In [1]:
from utils.load_dataset import *
from utils.custom_utils import *

In [2]:
model_path_var = "./models/checkpoint_train_virtual_and_real.pth"
model_path_vtr = "./models/checkpoint_train_real_over_virtual.pth"
model_path_r = "./models/checkpoint_train_real.pth"
model_path_v = "./models/checkpoint_train_virtual.pth"

In [3]:
def load_model(path):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # create a Faster R-CNN model without pre-trained
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)

    num_classes = 3 # wheat or not(background)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    # replace the pre-trained model's head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    checkpoint = torch.load(path, map_location=device)
    model.load_state_dict(checkpoint['model'])
    # load the trained weights
    #model.load_state_dict(torch.load(path, map_location=device))
    model.eval()

    # move model to the right device
    model.to(device)
    return model, device

In [4]:
def take_prediction(prediction, threshold):
    boxes = prediction['boxes'].tolist()
    labels = prediction['labels'].tolist()
    scores = prediction['scores'].tolist()
    if len(boxes) == 0:
        return [([0,0,0,0],1,0.)]
    
    res = [t for t in zip(boxes,labels,scores) if t[2]>threshold]
    if len(res) == 0:
        res = [([0,0,0,0],1,0.)]
    return res

In [5]:
def visualize_prediction(dataset, list_imgs, model, device, path=None, thr=0.7):
    for l in list_imgs:
        img,target = dataset[l]
        with torch.no_grad():
            prediction = model([img.to(device)])
        p = take_prediction(prediction[0],thr)
        for bb,label,score in p:
            if label == 1:
                color = "green"
                text = f"no fallen: {score:.3f}"
            else:
                color = "red"
                text = f"fallen: {score:.3f}"
            x0,y0,x1,y1 = bb
            im = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
            draw = ImageDraw.Draw(im)
            draw.rectangle(((x0, y0),(x1,y1)), outline=color, width=3)
            draw.text((x0, y0), text)
        if path is None:
            ImageShow.show(im)
        else:
            im.save(f'snapshots/{path}{l}.png')

In [6]:
def classifier_performance(dataset, model, device, fallen_or_not, tr = 0.7):
    tn, tp, fn, fp = 0,0,0,0
    for im,target in tqdm(dataset):
        gt_labels = target['labels'].tolist()
        with torch.no_grad():
            prediction = model([im.to(device)])
        p = take_prediction(prediction[0], tr)
        pred_labels = [l for _,l,_ in p]
        len_gt_lab = len(gt_labels)
        len_pred_lab = len(pred_labels)
        fall_gt = [i for i in gt_labels if i==fallen_or_not]
        num_fall_gt = len(fall_gt)
        num_no_fall_gt = len_gt_lab - num_fall_gt

        fall_pred = [i for i in pred_labels if i==fallen_or_not] 
        num_fall_pred = len(fall_pred)
        num_no_fall_pred = len_pred_lab - num_fall_pred

        if num_fall_gt == num_fall_pred:
            tp += num_fall_gt
        elif num_fall_gt > num_fall_pred:
            tp += num_fall_pred
            fn += (num_fall_gt - num_fall_pred)
        else:
            tp += num_fall_gt
            fp += (num_fall_pred - num_fall_gt)
    return tp, fp, fn

# UP-FALL DETECTION

In [7]:
test_set = load_data("test_set.txt")
test_dataset = FallenPeople(test_set, "test_imgs", FallenPeople.valid_test_transform())

In [8]:
no_fall = 0
fall = 0
for t in test_dataset:
    if t[1]['labels'][0] == 1:
        no_fall += 1
    elif t[1]['labels'][0] == 2:
        fall += 1
print(no_fall)
print(fall)

598
361


In [9]:
def collate_fn(batch):
        return tuple(zip(*batch))
    
test_data_loader = DataLoader(
      test_dataset,
      batch_size=4,
      shuffle=False,
      collate_fn=collate_fn
    )

In [10]:
model_var, device = load_model(model_path_var)
model_vtr, _ = load_model(model_path_vtr)
model_r, _ = load_model(model_path_r)
model_v, _ = load_model(model_path_v)
test_list = [random.randint(0,len(test_dataset)-1) for i in range(5)]

# ONLY REAL

In [11]:
evaluate(model_r, test_data_loader, device=device)

creating index...
index created!
Test:  [  0/240]  eta: 0:01:36  model_time: 0.3418 (0.3418)  evaluator_time: 0.0050 (0.0050)  time: 0.4001  data: 0.0501  max mem: 2025
Test:  [100/240]  eta: 0:00:53  model_time: 0.3279 (0.3284)  evaluator_time: 0.0040 (0.0043)  time: 0.3839  data: 0.0477  max mem: 2025
Test:  [200/240]  eta: 0:00:15  model_time: 0.3294 (0.3287)  evaluator_time: 0.0040 (0.0042)  time: 0.3841  data: 0.0471  max mem: 2025
Test:  [239/240]  eta: 0:00:00  model_time: 0.3300 (0.3288)  evaluator_time: 0.0039 (0.0042)  time: 0.3821  data: 0.0467  max mem: 2025
Test: Total time: 0:01:32 (0.3838 s / it)
Averaged stats: model_time: 0.3300 (0.3288)  evaluator_time: 0.0039 (0.0042)
Accumulating evaluation results...
DONE (t=0.24s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.490
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.949
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.

### FALL DETECTION

In [12]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_r, device, 2,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 959/959 [01:39<00:00,  9.66it/s]


TP: 391	FN: 0	FP: 523
Recall: 1.00
Precision: 0.43
F1-score: 0.60


100%|██████████| 959/959 [01:41<00:00,  9.44it/s]


TP: 391	FN: 0	FP: 253
Recall: 1.00
Precision: 0.61
F1-score: 0.76


100%|██████████| 959/959 [01:42<00:00,  9.39it/s]


...

TP: 374	FN: 17	FP: 11
Recall: 0.96
Precision: 0.97
F1-score: 0.96


100%|██████████| 959/959 [01:41<00:00,  9.42it/s]


TP: 371	FN: 20	FP: 5
Recall: 0.95
Precision: 0.99
F1-score: 0.97


100%|██████████| 959/959 [01:42<00:00,  9.33it/s]

TP: 355	FN: 36	FP: 3
Recall: 0.91
Precision: 0.99
F1-score: 0.95


In [13]:
print(recall)
print(precision)
print(f1_score)

['1.00', '1.00', '1.00', '0.98', '0.97', '0.97', '0.96', '0.96', '0.95', '0.91']
['0.43', '0.61', '0.79', '0.87', '0.91', '0.93', '0.95', '0.97', '0.99', '0.99']
['0.60', '0.76', '0.88', '0.92', '0.94', '0.95', '0.96', '0.96', '0.97', '0.95']


### NO-FALL DETECTION

In [14]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_r, device, 1,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 959/959 [01:38<00:00,  9.70it/s]


TP: 822	FN: 8	FP: 497
Recall: 0.99
Precision: 0.62
F1-score: 0.77


100%|██████████| 959/959 [01:40<00:00,  9.50it/s]


TP: 819	FN: 11	FP: 303
Recall: 0.99
Precision: 0.73
F1-score: 0.84


100%|██████████| 959/959 [01:41<00:00,  9.41it/s]


...

TP: 756	FN: 74	FP: 65
Recall: 0.91
Precision: 0.92
F1-score: 0.92


100%|██████████| 959/959 [01:42<00:00,  9.32it/s]


TP: 728	FN: 102	FP: 49
Recall: 0.88
Precision: 0.94
F1-score: 0.91


100%|██████████| 959/959 [01:41<00:00,  9.41it/s]

TP: 682	FN: 148	FP: 50
Recall: 0.82
Precision: 0.93
F1-score: 0.87


In [15]:
print(recall)
print(precision)
print(f1_score)

['0.99', '0.99', '0.98', '0.97', '0.96', '0.95', '0.93', '0.91', '0.88', '0.82']
['0.62', '0.73', '0.84', '0.89', '0.91', '0.92', '0.92', '0.92', '0.94', '0.93']
['0.77', '0.84', '0.90', '0.93', '0.93', '0.93', '0.92', '0.92', '0.91', '0.87']


## ONLY VIRTUAL

In [16]:
evaluate(model_v, test_data_loader, device=device)

creating index...
index created!
Test:  [  0/240]  eta: 0:01:37  model_time: 0.3293 (0.3293)  evaluator_time: 0.0059 (0.0059)  time: 0.4048  data: 0.0453  max mem: 2025
Test:  [100/240]  eta: 0:00:53  model_time: 0.3299 (0.3289)  evaluator_time: 0.0050 (0.0065)  time: 0.3864  data: 0.0471  max mem: 2025
Test:  [200/240]  eta: 0:00:15  model_time: 0.3329 (0.3305)  evaluator_time: 0.0041 (0.0058)  time: 0.3857  data: 0.0458  max mem: 2025
Test:  [239/240]  eta: 0:00:00  model_time: 0.3299 (0.3305)  evaluator_time: 0.0049 (0.0057)  time: 0.3821  data: 0.0463  max mem: 2025
Test: Total time: 0:01:32 (0.3868 s / it)
Averaged stats: model_time: 0.3299 (0.3305)  evaluator_time: 0.0049 (0.0057)
Accumulating evaluation results...
DONE (t=0.35s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.403
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.724
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.

### FALL DETECTION

In [17]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_v, device, 2,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 959/959 [01:40<00:00,  9.51it/s]


TP: 385	FN: 6	FP: 1700
Recall: 0.98
Precision: 0.18
F1-score: 0.31


100%|██████████| 959/959 [01:41<00:00,  9.43it/s]


TP: 370	FN: 21	FP: 751
Recall: 0.95
Precision: 0.33
F1-score: 0.49


100%|██████████| 959/959 [01:41<00:00,  9.47it/s]


...

TP: 119	FN: 272	FP: 12
Recall: 0.30
Precision: 0.91
F1-score: 0.46


100%|██████████| 959/959 [01:41<00:00,  9.46it/s]


TP: 49	FN: 342	FP: 0
Recall: 0.13
Precision: 1.00
F1-score: 0.22


100%|██████████| 959/959 [01:41<00:00,  9.49it/s]


ZeroDivisionError: division by zero

In [18]:
print(recall)
print(precision)
print(f1_score)

['0.98', '0.95', '0.87', '0.75', '0.65', '0.46', '0.30', '0.13']
['0.18', '0.33', '0.47', '0.59', '0.73', '0.83', '0.91', '1.00']
['0.31', '0.49', '0.61', '0.66', '0.69', '0.59', '0.46', '0.22']


### NO-FALL DETECTION

In [19]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_v, device, 1,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 959/959 [01:40<00:00,  9.58it/s]


TP: 828	FN: 2	FP: 2845
Recall: 1.00
Precision: 0.23
F1-score: 0.37


100%|██████████| 959/959 [01:40<00:00,  9.53it/s]


TP: 825	FN: 5	FP: 1898
Recall: 0.99
Precision: 0.30
F1-score: 0.46


100%|██████████| 959/959 [01:40<00:00,  9.50it/s]


...

TP: 792	FN: 38	FP: 391
Recall: 0.95
Precision: 0.67
F1-score: 0.79


100%|██████████| 959/959 [01:41<00:00,  9.40it/s]


TP: 781	FN: 49	FP: 386
Recall: 0.94
Precision: 0.67
F1-score: 0.78


100%|██████████| 959/959 [01:40<00:00,  9.50it/s]

TP: 750	FN: 80	FP: 348
Recall: 0.90
Precision: 0.68
F1-score: 0.78


In [20]:
print(recall)
print(precision)
print(f1_score)

['1.00', '0.99', '0.99', '0.98', '0.96', '0.96', '0.96', '0.95', '0.94', '0.90']
['0.23', '0.30', '0.40', '0.48', '0.55', '0.61', '0.65', '0.67', '0.67', '0.68']
['0.37', '0.46', '0.57', '0.65', '0.70', '0.75', '0.78', '0.79', '0.78', '0.78']


## VIRTUAL THAN REAL

In [21]:
evaluate(model_vtr, test_data_loader, device=device)

creating index...
index created!
Test:  [  0/240]  eta: 0:01:35  model_time: 0.3329 (0.3329)  evaluator_time: 0.0037 (0.0037)  time: 0.3984  data: 0.0454  max mem: 2025
Test:  [100/240]  eta: 0:00:53  model_time: 0.3406 (0.3307)  evaluator_time: 0.0044 (0.0043)  time: 0.3936  data: 0.0462  max mem: 2025
Test:  [200/240]  eta: 0:00:15  model_time: 0.3379 (0.3350)  evaluator_time: 0.0034 (0.0042)  time: 0.3952  data: 0.0486  max mem: 2025
Test:  [239/240]  eta: 0:00:00  model_time: 0.3405 (0.3355)  evaluator_time: 0.0038 (0.0042)  time: 0.3911  data: 0.0480  max mem: 2025
Test: Total time: 0:01:33 (0.3906 s / it)
Averaged stats: model_time: 0.3405 (0.3355)  evaluator_time: 0.0038 (0.0042)
Accumulating evaluation results...
DONE (t=0.21s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.410
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.843
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.

### FALL DETECTION

In [22]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_vtr, device, 2,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 959/959 [01:42<00:00,  9.35it/s]


TP: 388	FN: 3	FP: 851
Recall: 0.99
Precision: 0.31
F1-score: 0.48


100%|██████████| 959/959 [01:42<00:00,  9.34it/s]


TP: 387	FN: 4	FP: 518
Recall: 0.99
Precision: 0.43
F1-score: 0.60


100%|██████████| 959/959 [01:42<00:00,  9.39it/s]


...

TP: 354	FN: 37	FP: 61
Recall: 0.91
Precision: 0.85
F1-score: 0.88


100%|██████████| 959/959 [01:42<00:00,  9.34it/s]


TP: 334	FN: 57	FP: 33
Recall: 0.85
Precision: 0.91
F1-score: 0.88


100%|██████████| 959/959 [01:42<00:00,  9.39it/s]

TP: 305	FN: 86	FP: 13
Recall: 0.78
Precision: 0.96
F1-score: 0.86


In [23]:
print(recall)
print(precision)
print(f1_score)

['0.99', '0.99', '0.99', '0.98', '0.97', '0.95', '0.93', '0.91', '0.85', '0.78']
['0.31', '0.43', '0.57', '0.66', '0.74', '0.79', '0.83', '0.85', '0.91', '0.96']
['0.48', '0.60', '0.73', '0.79', '0.84', '0.86', '0.88', '0.88', '0.88', '0.86']


### NO-FALL DETECTION

In [24]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_vtr, device, 1,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 959/959 [01:41<00:00,  9.48it/s]


TP: 774	FN: 56	FP: 455
Recall: 0.93
Precision: 0.63
F1-score: 0.75


100%|██████████| 959/959 [01:42<00:00,  9.35it/s]


TP: 744	FN: 86	FP: 265
Recall: 0.90
Precision: 0.74
F1-score: 0.81


100%|██████████| 959/959 [01:42<00:00,  9.38it/s]


...

TP: 675	FN: 155	FP: 30
Recall: 0.81
Precision: 0.96
F1-score: 0.88


100%|██████████| 959/959 [01:42<00:00,  9.36it/s]


TP: 667	FN: 163	FP: 40
Recall: 0.80
Precision: 0.94
F1-score: 0.87


100%|██████████| 959/959 [01:42<00:00,  9.35it/s]

TP: 639	FN: 191	FP: 61
Recall: 0.77
Precision: 0.91
F1-score: 0.84


In [25]:
print(recall)
print(precision)
print(f1_score)

['0.93', '0.90', '0.87', '0.86', '0.85', '0.84', '0.83', '0.81', '0.80', '0.77']
['0.63', '0.74', '0.83', '0.91', '0.96', '0.97', '0.96', '0.96', '0.94', '0.91']
['0.75', '0.81', '0.85', '0.88', '0.90', '0.90', '0.89', '0.88', '0.87', '0.84']


## VIRTUAL AND REAL

In [26]:
evaluate(model_var, test_data_loader, device=device)

creating index...
index created!
Test:  [  0/240]  eta: 0:01:33  model_time: 0.3270 (0.3270)  evaluator_time: 0.0037 (0.0037)  time: 0.3915  data: 0.0358  max mem: 2025
Test:  [100/240]  eta: 0:00:52  model_time: 0.3204 (0.3197)  evaluator_time: 0.0047 (0.0053)  time: 0.3766  data: 0.0479  max mem: 2025
Test:  [200/240]  eta: 0:00:15  model_time: 0.3299 (0.3230)  evaluator_time: 0.0044 (0.0052)  time: 0.3834  data: 0.0456  max mem: 2025
Test:  [239/240]  eta: 0:00:00  model_time: 0.3344 (0.3243)  evaluator_time: 0.0048 (0.0053)  time: 0.3860  data: 0.0469  max mem: 2025
Test: Total time: 0:01:31 (0.3806 s / it)
Averaged stats: model_time: 0.3344 (0.3243)  evaluator_time: 0.0048 (0.0053)
Accumulating evaluation results...
DONE (t=0.19s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.512
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.941
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.

### FALL DETECTION

In [27]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_var, device, 2,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 959/959 [01:38<00:00,  9.71it/s]


TP: 389	FN: 2	FP: 1392
Recall: 0.99
Precision: 0.22
F1-score: 0.36


100%|██████████| 959/959 [01:41<00:00,  9.43it/s]


TP: 388	FN: 3	FP: 628
Recall: 0.99
Precision: 0.38
F1-score: 0.55


100%|██████████| 959/959 [01:41<00:00,  9.46it/s]


...

TP: 357	FN: 34	FP: 11
Recall: 0.91
Precision: 0.97
F1-score: 0.94


100%|██████████| 959/959 [01:42<00:00,  9.39it/s]


TP: 344	FN: 47	FP: 2
Recall: 0.88
Precision: 0.99
F1-score: 0.93


100%|██████████| 959/959 [01:41<00:00,  9.45it/s]

TP: 293	FN: 98	FP: 1
Recall: 0.75
Precision: 1.00
F1-score: 0.86


In [28]:
print(recall)
print(precision)
print(f1_score)

['0.99', '0.99', '0.98', '0.97', '0.96', '0.96', '0.94', '0.91', '0.88', '0.75']
['0.22', '0.38', '0.74', '0.81', '0.87', '0.91', '0.95', '0.97', '0.99', '1.00']
['0.36', '0.55', '0.84', '0.88', '0.91', '0.94', '0.95', '0.94', '0.93', '0.86']


### NO-FALL DETECTION

In [29]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_var, device, 1,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 959/959 [01:39<00:00,  9.65it/s]


TP: 826	FN: 4	FP: 1274
Recall: 1.00
Precision: 0.39
F1-score: 0.56


100%|██████████| 959/959 [01:41<00:00,  9.42it/s]


TP: 819	FN: 11	FP: 596
Recall: 0.99
Precision: 0.58
F1-score: 0.73


100%|██████████| 959/959 [01:42<00:00,  9.37it/s]


...

TP: 749	FN: 81	FP: 33
Recall: 0.90
Precision: 0.96
F1-score: 0.93


100%|██████████| 959/959 [01:42<00:00,  9.38it/s]


TP: 737	FN: 93	FP: 41
Recall: 0.89
Precision: 0.95
F1-score: 0.92


100%|██████████| 959/959 [01:41<00:00,  9.43it/s]

TP: 652	FN: 178	FP: 82
Recall: 0.79
Precision: 0.89
F1-score: 0.83


In [30]:
print(recall)
print(precision)
print(f1_score)

['1.00', '0.99', '0.96', '0.93', '0.93', '0.92', '0.91', '0.90', '0.89', '0.79']
['0.39', '0.58', '0.75', '0.92', '0.95', '0.97', '0.97', '0.96', '0.95', '0.89']
['0.56', '0.73', '0.85', '0.92', '0.94', '0.94', '0.94', '0.93', '0.92', '0.83']
